In [111]:
import boto3
import base64
from botocore.exceptions import ClientError


import re
import json
import logging
import os
import pg
import pgdb

logger = logging.getLogger()
logger.setLevel(logging.INFO)
from pprint import pprint


In [26]:
def using_meta():
    """How to get to resource object through client object
    Paremeter: opt(int)
    return: None
    """
    resource = boto3.session.Session().resource('ec2', 'us-east-1')
    for each_region in resource.meta.client.describe_regions()['Regions']:
        print(each_region.get('RegionName'))
    return None

using_meta()

eu-north-1
ap-south-1
eu-west-3
eu-west-2
eu-west-1
ap-northeast-3
ap-northeast-2
ap-northeast-1
sa-east-1
ca-central-1
ap-southeast-1
ap-southeast-2
eu-central-1
us-east-1
us-east-2
us-west-1
us-west-2


## Colection object 

In [37]:
def collection_objects():
    """How to get to resource object through client object
    Paremeter: opt(int)
    return: None
    """
    resource = boto3.session.Session().resource(service_name='ec2', region_name='us-east-1')
    pprint(dir(resource.instances))
    
    
collection_objects()    

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_collection_cls',
 '_handler',
 '_model',
 '_parent',
 'all',
 'create_tags',
 'filter',
 'iterator',
 'limit',
 'monitor',
 'page_size',
 'pages',
 'reboot',
 'start',
 'stop',
 'terminate',
 'unmonitor']


## If you look from the above method 
#### there's Filter, all and limit
### These are your collection object 

## Here we are creating a function to using colllection method of all()

In [40]:
def collection_objects():
    """How to get to resource object through client object
    Paremeter: opt(int)
    return: None
    """
    resource = boto3.session.Session().resource(service_name='ec2', region_name='us-east-1')
    for each_instance in resource.instances.all():
        print(each_instance)
    
    
collection_objects() 

ec2.Instance(id='i-0402675382b992f7a')
ec2.Instance(id='i-089999e2cced39b86')
ec2.Instance(id='i-09135aa74c2e472d7')


### Let's using limit() collection method

In [41]:
def collection_objects():
    """How to get to resource object through client object
    Paremeter: opt(int)
    return: None
    """
    resource = boto3.session.Session().resource(service_name='ec2', region_name='us-east-1')
    for each_instance in resource.instances.limit(2):
        print(each_instance)
    
    
collection_objects() 

ec2.Instance(id='i-0402675382b992f7a')
ec2.Instance(id='i-089999e2cced39b86')


### Now let's see Filter()

In [42]:
def collection_objects():
    """How to get to resource object through client object
    Paremeter: opt(int)
    return: None
    """
    resource = boto3.session.Session().resource(service_name='ec2', region_name='us-east-1')
    for each_instance in resource.instances.filter(Filters=[]):
        print(each_instance)
    
    
collection_objects() 

ec2.Instance(id='i-0402675382b992f7a')
ec2.Instance(id='i-089999e2cced39b86')
ec2.Instance(id='i-09135aa74c2e472d7')


### Filter have one parameter which is a list. and in that list we have a dictionary

let's create a filer block. to get all running instances

In [46]:
filter_1 = {"Name": "instance-state-name", "Values": ['terminated']}

def collection_objects():
    """How to get to resource object through client object
    Paremeter: opt(int)
    return: None
    """
    resource = boto3.session.Session().resource(service_name='ec2', region_name='us-east-1')
    for each_instance in resource.instances.filter(Filters=[filter_1]):
        print(each_instance)
    
    
collection_objects()

ec2.Instance(id='i-0402675382b992f7a')
ec2.Instance(id='i-089999e2cced39b86')
ec2.Instance(id='i-09135aa74c2e472d7')


We can filter on list

## Boto3 Collection Concept to start, stop, terminate, monito all instances with a single shut

In [38]:
def ec2_instance_ids():
    ids = []
    running_instance = {"Name": "instance-state-name", "Values":["running"]}
    resource = boto3.session.Session(profile_name="sbx").resource(service_name='ec2', region_name='us-east-1')
    for each_id in resource.instances.all():
        ids.append(each_id.id)
    return ids 

ids = ec2_instance_ids()


def start_instances_waiter():
    """How to get to resource object through client object
    Paremeter: opt(int)
    return: None
    """
    resource = boto3.session.Session(profile_name="sbx").resource(service_name='ec2', region_name='us-east-1')
    client = boto3.session.Session(profile_name="sbx").client(service_name='ec2', region_name='us-east-1')
    try:
        waiter = client.get_waiter('instance_running') 
        print("starting all instances....")
        resource.instances.start()
        waiter.wait(InstanceIds=ids)
        print("starting all instances!!...")
    except botocore.exceptions.ClientError as err:
        if err.response.get('Error')['Code'] == 'IncorrectInstanceState':
            print("Oooops, could not be started at this time")
        else:
            print(err.response.get('Error')['Code'])
    except Exception as err:
        print("Oooops, could not be started at this time")
    
start_instances_waiter()

['i-09c94869e20bb9d3b', 'i-09b52b6883f6b122a', 'i-0349cb744bae81bc8', 'i-0be42db74bc1e74a7']
starting all instances....
Oooops, could not be started at this time


## It is always better to use a client waiters

In [41]:
def stop_instances_waiter():
    """How to get to resource object through client object
    Paremeter: opt(int)
    return: None
    """
    resource = boto3.session.Session(profile_name="sbx").resource(service_name='ec2', region_name='us-east-1')
    client = boto3.session.Session(profile_name="sbx").client(service_name='ec2', region_name='us-east-1')
    try:
        waiter = client.get_waiter('instance_stopped') 
        print("stopping all instances....")
        resource.instances.stop()
        waiter.wait(InstanceIds=ids)
        print("stopping all instances!!...")
    except botocore.exceptions.ClientError as err:
        if err.response.get('Error')['Code'] == 'IncorrectInstanceState':
            print("Oooops, could not be started at this time")
        else:
            raise err.response.get('Error')['Code']
    except Exception as err:
        print("Oooops, could not be started at this time")    
    
stop_instances_waiter()

stopping all instances....
Oooops, could not be started at this time


In [42]:
def terminate_instances_waiter():
    """How to get to resource object through client object
    Paremeter: opt(int)
    return: None
    """
    resource = boto3.session.Session(profile_name="sbx").resource(service_name='ec2', region_name='us-east-1')
    client = boto3.session.Session(profile_name="sbx").client(service_name='ec2', region_name='us-east-1')
    waiter = client.get_waiter('instance_terminated') 
    print("stopping all instances....")
    resource.instances.terminate()
    waiter.wait(InstanceIds=ids)
    print("stopping all instances!!...")
    
terminate_instances_waiter()    

stopping all instances....
stopping all instances!!...


In [25]:
print(dir(botocore.exceptions.ClientError))

['MSG_TEMPLATE', '__cause__', '__class__', '__context__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__suppress_context__', '__traceback__', '__weakref__', '_get_retry_info', 'args', 'with_traceback']


In [44]:
print(ec2_instance_ids())

['i-09c94869e20bb9d3b', 'i-09b52b6883f6b122a', 'i-0349cb744bae81bc8', 'i-0be42db74bc1e74a7']


In [72]:
def delete_untaged_ebs():
    """This function is used to delete untaged volumes
    Parameter: None
    return
    """
    try:
        client = boto3.session.Session(profile_name="sbx").client(service_name='ec2', region_name='us-east-1')
    except botocore.exceptions.ProfileNotFound as err:
        print("profile not found")
    except botocore.exceptions.UnknownServiceError:
        print("unkown service")
    except botocore.exceptions.InvalidRegionError:
        print("Region not found. please check your region and try and again")
    else:
        print("No exception found")
   
        

    
        

    
    
delete_untaged_ebs()    
        
    

Ending exception


In [139]:


def working_with_secrets():
    secrets_name = "python-boto3"
    client = boto3.session.Session(profile_name="shared").client(service_name="secretsmanager", region_name="us-east-1")
    
    try:
        secrets_response = client.get_secret_value(SecretId=secrets_name)
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
    else:
        if 'SecretString' in secrets_response:
            plaintext = secrets_response['SecretString']
            secret_dict = json.loads(plaintext)
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
    
    return secret_dict
    

    
secrets = working_with_secrets()    

print(secrets['password'])
    

P@ssword


## describe secrets.

In [120]:
def describe_secrets():
    client = boto3.session.Session(profile_name="shared").client(service_name="secretsmanager", region_name="us-east-1")
    secret_arn = 'arn:aws:secretsmanager:us-east-1:181437319056:secret:python-boto3-lUMM6t'
    metadata = client.describe_secret(SecretId=secret_arn)
    pprint(metadata)
    if 'RotationEnabled' in metadata and not metadata['RotationEnabled']:
        logger.error("Secrets rotation is not enabled",secret_arn )
        raise ValueError("Secrets rotation is not enabled",secret_arn)
    version = metadata['VersionIdsToStages']
    print(version)
    if token not in versions:
        logger.error("Secret version %s has no stage for rotation of secret %s." % (token, arn))
        raise ValueError("Secret version %s has no stage for rotation of secret %s." % (token, arn)) 
    
    
describe_secrets()    


{'ARN': 'arn:aws:secretsmanager:us-east-1:181437319056:secret:python-boto3-lUMM6t',
 'CreatedDate': datetime.datetime(2022, 6, 26, 17, 21, 56, 456000, tzinfo=tzlocal()),
 'Description': 'This secrets is use to test python boto3',
 'LastAccessedDate': datetime.datetime(2022, 6, 26, 17, 0, tzinfo=tzlocal()),
 'LastChangedDate': datetime.datetime(2022, 6, 26, 17, 21, 56, 502000, tzinfo=tzlocal()),
 'Name': 'python-boto3',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '344',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Mon, 27 Jun 2022 04:19:17 GMT',
                                      'x-amzn-requestid': 'aea3bb88-5875-4786-bec5-c5d6a4188f1c'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'aea3bb88-5875-4786-bec5-c5d6a4188f1c',
                      'RetryAttempts': 0},
 'Tags': [],
 'VersionIdsToStages': {'e4677bb4-fdfe-48f0-96db-29f25bf9aff3': ['AWSCURRENT

In [143]:
dir(botocore.exceptions.ValidationError)

['__cause__',
 '__class__',
 '__context__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__suppress_context__',
 '__traceback__',
 '__weakref__',
 'args',
 'fmt',
 'with_traceback']